In [313]:
import pandas as pd
import numpy as np
original='/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/B1_1.csv'
result_txt=original[:-4]+".txt"
result_eda_csv=original[:-4]+"_1eda.csv"
print(result_txt)
#print()
df = pd.read_csv(original,engine='python')
df=df[0:7000]


/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/B1_1.txt


In [314]:
df['label_category'].value_counts()
# 2 含有贬义的言辞(或行为)
# 3 仇恨; 愤怒; 敌意; 憎恶;
# 4 偏见的讨论
# 1 威胁，计划伤害和煽动

none                                        5614
2. derogation                                697
3. animosity                                 462
4. prejudiced discussions                    123
1. threats, plans to harm and incitement     104
Name: label_category, dtype: int64

In [315]:
# df = pd.read_csv('train_all_tasks_eda.csv',engine='python')
# df.info()
# df['label'].value_counts()

In [316]:
# import pandas as pd

# 将training_all_tasks_new.csv改为EDA所需的txt文本
# 数据格式为标签\t文本
from nltk.corpus import stopwords

def change_to_txt(csvname,result_txt):
    data = pd.read_csv(csvname, encoding='UTF-8')
    #data=data[data['label_category']!='none']
    #data=data[:7000]
    id = data.iloc[:, 0].tolist()
    text = data.iloc[:, 13].tolist()
    label_category= data.iloc[:, 3].tolist()
    
    text_stop=[]
    for i in text:
        words = i.split(' ')
        words = [word for word in words if word != '']
        #words=[str(word) for word in words if word not in stopwords.words('english')]
        #print(" ".join([word for word in words if word not in stopwords]))
        text_stop.append(" ".join(words))


 
    label = []
    for i in label_category:
        # if i == '1. threats, plans to harm and incitement':
        #     label.append('1')
        # elif i == '2. derogation':
        #     label.append('2')
        # elif i == '3. animosity':
        #     label.append('3')
        # elif i == '4. prejudiced discussions':
        #     label.append('4')
        if i=="sexist":
            label.append('1')
        elif i=="not sexist":
            label.append('0')

    data_all = []
    for j in range(len(id)):
        #print(j)
        #print(label[j])
        #print(id[j])
        #print(text_stop[j])
        a = label[j] +'\t'+ id[j] + '\t' + text_stop[j]
        data_all.append(a)
    f = open(result_txt, 'w', encoding='UTF-8')
    for k in data_all:
        f.write(k+'\n')
    f.close()

In [317]:
# Easy data augmentation techniques for text classification
# Jason Wei and Kai Zou

import random
from random import shuffle
random.seed(1)
# for the first time you use wordnet
# import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet

# stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '', '[URL]']


# cleaning up text
import re


def get_only_chars(line):
	clean_line = ""
	line = line.replace("’", "")
	line = line.replace("'", "")
	line = line.replace("-", " ")  #r eplace hyphens with spaces
	line = line.replace("\t", " ")
	line = line.replace("\n", " ")
	line = line.lower()
	for char in line:
		if char in 'qwertyuiopasdfghjklzxcvbnm ':
			clean_line += char
		else:
			clean_line += ' '
	clean_line = re.sub(' +',' ', clean_line)  # delete extra spaces
	if clean_line[0] == ' ':
		clean_line = clean_line[1:]
	return clean_line

########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################


def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			# print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: # only replace up to n words
			break

	# this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words


def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################


def random_deletion(words, p):

	# obviously, if there's only one word, don't delete it
	if len(words) == 1:
		return words

	# randomly delete words with probability p
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	# if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################


def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words


def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################


def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		random_word = new_words[random.randint(0, len(new_words)-1)]
		synonyms = get_synonyms(random_word)
		counter += 1
		if counter >= 10:
			return
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

########################################################################
# main data augmentation function
########################################################################


def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	
	sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word != '']
	#print(words)
	#words=[word for word in words if word not in stop_words]
	num_words = len(words)
	
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	# sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	# ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	# rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	# rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	# trim so that we have the desired number of augmented sentences
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	# append the original sentence
	augmented_sentences.append(sentence)
	return augmented_sentences


In [318]:
#from eda代码.eda import *
import pandas as pd
# arguments to be parsed from command line
import argparse
ap = argparse.ArgumentParser()
#ap.add_argument("--input", required=True, type=str, help="input file of unaugmented data")
ap.add_argument("--input", type=str, help="input file of unaugmented data", default=result_txt)
ap.add_argument("--output", required=False, type=str, help="output file of unaugmented data")
ap.add_argument("--num_aug", required=False, type=int, help="number of augmented sentences per original sentence")
ap.add_argument("--alpha_sr", required=False, type=float, help="percent of words in each sentence to be replaced by synonyms")
ap.add_argument("--alpha_ri", required=False, type=float, help="percent of words in each sentence to be inserted")
ap.add_argument("--alpha_rs", required=False, type=float, help="percent of words in each sentence to be swapped")
ap.add_argument("--alpha_rd", required=False, type=float, help="percent of words in each sentence to be deleted")
#args = ap.parse_args()
args = ap.parse_args(args=[])

# the output file
output = None
if args.output:
    output = args.output
else:
    from os.path import dirname, basename, join
    output = join(dirname(args.input), 'eda_' + basename(args.input))

# number of augmented sentences to generate per original sentence
# num_aug = 9  # default
num_aug = 6  # 每条数据再扩充4条
if args.num_aug:
    num_aug = args.num_aug

# how much to replace each word by synonyms
alpha_sr = 0.1  # default
if args.alpha_sr is not None:
    alpha_sr = args.alpha_sr

# how much to insert new words that are synonyms
alpha_ri = 0.1  # default
if args.alpha_ri is not None:
    alpha_ri = args.alpha_ri

# how much to swap words
alpha_rs = 0.1  # default
if args.alpha_rs is not None:
    alpha_rs = args.alpha_rs

# how much to delete words
alpha_rd = 0  # default=0.1
if args.alpha_rd is not None:
    alpha_rd = args.alpha_rd

if alpha_sr == alpha_ri == alpha_rs == alpha_rd == 0:
     ap.error('At least one alpha should be greater than zero')


# # generate more data with standard augmentation
def gen_eda(train_orig, output_file, alpha_sr, alpha_ri, alpha_rs, alpha_rd, num_aug):

    writer = open(output_file, 'w', encoding='UTF-8')
    lines = open(train_orig, 'r', encoding='UTF-8').readlines()

    for i, line in enumerate(lines):
        parts = line[:-1].split('\t')
        id=parts[1]
        label = parts[0]
        sentence = parts[2]
        aug_sentences = eda(sentence, alpha_sr=alpha_sr, alpha_ri=alpha_ri, alpha_rs=alpha_rs, p_rd=alpha_rd, num_aug=num_aug)
        for aug_sentence in aug_sentences:
            writer.write(label+"\t" + aug_sentence + '\t' + id + "\n")

    writer.close()
    print("generated augmented sentences with eda for " + train_orig + " to " + output_file + " with num_aug=" + str(num_aug))
    return output_file


def txt_adjust(txt):
    # 将EDA后的txt文件转换为csv，需对应修改label及输出文件名
    file = open(txt, 'r', encoding='UTF-8')
    data = file.readlines()
    all = []
    label = []
    text = []
    id=[]
    # label = []
    for line in data:
        f = line.split('\t')  # 将每行数据按\t重新划分为标签和数据
        all.append(f)
    for i in all:
        #print(i)
        #print(i[1])
        i[1] = i[1].replace('\n', '')
        i[2] = i[2].replace('\n', '')
        label.append(i[0])
        text.append(i[1])
        id.append(i[2])
    file.close()
    df = pd.DataFrame()
    df['label'] = label
    df['text'] = text
    df['rewire_id'] = id
    # df['label'] = label
    return df

In [319]:
# main function
if __name__ == "__main__":
    # 在configuration的parameter中添加 --input=需要数据增强的txt文件名
    # generate augmented sentences and output into a new file
    change_to_txt(original,result_txt)
    txt = gen_eda(args.input, output, alpha_sr=alpha_sr, alpha_ri=alpha_ri,
                  alpha_rs=alpha_rs, alpha_rd=alpha_rd, num_aug=num_aug)
    file = txt_adjust(txt)
    file.to_csv(result_eda_csv, index=False, encoding='UTF-8')

generated augmented sentences with eda for /Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/B1_1.txt to /Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/eda_B1_1.txt with num_aug=6


In [320]:
df_0=pd.read_csv("/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/B0_1_eda.csv",engine='python')
df_0.info()
df_1=pd.read_csv("/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/B1_1_1eda.csv",engine='python')
df_1.info()
# df_2=pd.read_csv("/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/B1_1_2eda.csv",engine='python')
# df_2.info()
h= pd.concat([df_0,df_1],axis=0)
h.to_csv("/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/gold_4_6.csv", index=False, encoding='UTF-8')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13332 entries, 0 to 13331
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      13332 non-null  int64 
 1   text       13332 non-null  object
 2   rewire_id  13332 non-null  object
dtypes: int64(1), object(2)
memory usage: 312.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108892 entries, 0 to 108891
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   label      108892 non-null  int64 
 1   text       108892 non-null  object
 2   rewire_id  108892 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.5+ MB


In [322]:
df_14=pd.read_csv('/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/C/train_all_tasks.csv',engine='python')
df = pd.read_csv('/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/gold_4_6.csv',engine='python')

file1=np.array(df_14)
x=[x[0] for x in file1]

df=df[df["rewire_id"].isin(x)]
df.info()
df.to_csv("/Users/tianyumi/Desktop/Projects/EDOS/Edos_Coding/EDA/gold/gold_3_6.csv", index=False, encoding='UTF-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85400 entries, 0 to 122216
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      85400 non-null  int64 
 1   text       85400 non-null  object
 2   rewire_id  85400 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.6+ MB
